# Homework 05. Network Flow/Transportation

MSCA 32013 Optimization and Simulation Methods. University of Chicago.

Group 7. Ben Ossyra, Peter Pezon. Yawen Zhang. 

## Problem 1
Consider the network flow in the diagram below. Each circle contains a number representing a location. Each diamond shape contains a number representing the supply or demand for a given commodity at each location. Each octagon shape contains a number representing the cost (per unit) for transporting the commodity from one location to another. Each arrows represent the direction that the commodity can flow from a location to another.

Each row represents each path, and column c_ij represents the cost per unit on corresponding path:

| Start_node_i | End_node_j | c_ij |
| ----- | ------ | ----- | 
|  1    |	2	 |	2	 | 
|  1    |	3	 |	-5 | 
|  2    |	3	 |	-1 | 
|  2    |	4	 |	4 | 
|  3    |	4	 |	3 | 
|  3    |	2	 |	6 |
|  4    |	1	 |	7 |

#### Define our variables
There are 7 paths, so we have 7 variables:

* $x_{12}$ is the number of units that transported from location 1 to 2.
* $x_{13}$ is the number of units that transported from location 1 to 3.
* $x_{23}$ is the number of units that transported from location 2 to 3.
* $x_{24}$ is the number of units that transported from location 2 to 4.
* $x_{34}$ is the number of units that transported from location 3 to 4.
* $x_{32}$ is the number of units that transported from location 3 to 2.
* $x_{41}$ is the number of units that transported from location 4 to 1.

#### Regarding constraints:

For each location, the total units transported from the location minus the total units transported to the location must be equal to the supply (or demand in the case of a negative number):

$$ x12 + x13 - x41 = 4 $$
$$ x23 + x24 - x12 = 2 $$
$$ x34 + x32 -x23 = -1 $$
$$ x41 + x24 -x34 = -5 $$



#### The objective function is defined as:

$$ min\ \  2x_{12} - 5x_{13} -1 x_{23} + 4x_{24} + 3x_{34} + 6x_{32} + 7x_{41} $$



In [10]:
using CSV, DataFrames

In [11]:
df = CSV.read("/Users/benjaminossyra/Desktop/Optimization/Homework_5/Network_Loc_Cost.csv", DataFrame; delim = ',', header=true)

Row,Start Node,End Node,c_ij
,Int64,Int64,Int64
1,1,2,2
2,1,3,-5
3,2,3,-1
4,2,4,4
5,3,2,6
6,3,4,3
7,4,1,7


In [12]:
df2 = CSV.read("/Users/benjaminossyra/Desktop/Optimization/Homework_5/Network_Supp_Dem.csv", DataFrame; delim = ',', header=true)

Row,Node,Supply_Demand
,Int64,Int64
1,1,4
2,2,2
3,3,-1
4,4,-5


In [13]:
using JuMP, GLPK

myModel = Model(GLPK.Optimizer)
@variable(myModel, x12>= 0)
@variable(myModel, x13>= 0)
@variable(myModel, x23>= 0)
@variable(myModel, x24>= 0)
@variable(myModel, x34>= 0)
@variable(myModel, x32>= 0)
@variable(myModel, x41>= 0)
@constraint(myModel, x12+x13-x41==4)
@constraint(myModel, x23+x24-x12-x32==2)
@constraint(myModel, x34+x32-x23==-1)
@constraint(myModel, x41-x24-x34==-5)


@objective(myModel, Min, 2*x12-5*x13-1*x23+4*x24+3*x34+6*x32+7*x41)
myModel



A JuMP Model
Minimization problem with:
Variables: 7
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.EqualTo{Float64}`: 4 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 7 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: GLPK
Names registered in the model: x12, x13, x23, x24, x32, x34, x41

In [14]:
print(myModel)

In [15]:
@time begin
    status = optimize!(myModel)
end
println("Objective value: ", JuMP.objective_value(myModel))
println("x12 = ", JuMP.value(x12))
println("x13 = ", JuMP.value(x13))
println("x23 = ", JuMP.value(x23))
println("x24 = ", JuMP.value(x24))
println("x34 = ", JuMP.value(x34))
println("x32 = ", JuMP.value(x32))
println("x41 = ", JuMP.value(x41))

  0.133904 seconds (422.43 k allocations: 22.840 MiB, 20.41% gc time, 99.30% compilation time: 100% of which was recompilation)
Objective value: 17.0
x12 = 4.0
x13 = 0.0
x23 = 6.0
x24 = 0.0
x34 = 5.0
x32 = 0.0
x41 = 0.0


#### Optimal solution: 
Transport 4 units from location 1 to 2, transport 6 units from location 2 to 3, and  transport 5 units from location 3 to 4.  In this way, the total cost is 17.

#### Further Explanation

Node 1 starts with 4 units. We transfer all 4 units to node 2 at a cost of 2/per unit.

Cost of Node 1 -> Node 2 = 2 * 4 = 8

Node 2 starts with 2 units and was transferred 4 units. There are now 6 units. All 6 units are transferred to node 3 at a cost of -1/per unit.

Cost of Node 2 -> Node 3 = 6 * -1 = -6

Node 3 starts with no units and was transffered 6 units. There are now 6 units. It has a demand of 1 unit, so we transfer 5 units (leaving 1) to node 4 at a cost of 3/unit.

Cost of Node 3 -> Node 4 = 5 * 3 = 15

Node 4 starts with no units and was transferred 5 units. It had a demand of 5 units that is now satisfied.

The total cost of moving the units is 8 + -6 + 15 = 17

#### Constraint Verification

constraint 1:  
$ x12 + x13 - x41 = 4 $  
$ 4 + 0 - 0 = 4 $  
constraint 1 is confirmed

constraint 2:  
$ x23 + x24 - x12 = 2 $  
$ 6 + 0 - 4 = 2 $  
constraint 2 is confirmed

constraint 3:  
$ x34 + x32 -x23 = -1 $  
$ 5 + 0 - 6 = -1 $  
constraint 3 is confirmed

constraint 4:  
$ x41 + x24 -x34 = -5 $  
$ 0 + 0 - 5 = -5 $  
constraint 4 is confirmed

# Problem 2 - Transportation

In [16]:
t = CSV.read("/Users/benjaminossyra/Desktop/Optimization/Homework_5/transportation.csv", DataFrame; delim = ',', header=true)

Row,Column1,R1,R2,R3,R4,R5,R6,R7,Supply
,String7,Int64,Int64?,Int64,Int64?,Int64,Int64,Int64,Int64?
1,SA,78,28,22,28,32,164,16,2800
2,SB,54,missing,24,missing,52,190,34,5200
3,SC,48,28,34,26,56,198,40,5800
4,Demand,1800,2400,1200,800,3400,2200,2000,missing


In [17]:
tm = Matrix(t)

4×9 Matrix{Any}:
 "SA"        78    28           22  …    32   164    16  2800
 "SB"        54      missing    24       52   190    34  5200
 "SC"        48    28           34       56   198    40  5800
 "Demand"  1800  2400         1200     3400  2200  2000      missing

Create a vector of the demand values

In [18]:
demand = tm[4,2:8]

7-element Vector{Any}:
 1800
 2400
 1200
  800
 3400
 2200
 2000

Create a vector of the supply values

In [19]:
supply = tm[1:3,9]

3-element Vector{Any}:
 2800
 5200
 5800

We are going to create a vector of the costs, and exclude the missing values since those routes are not possible

In [20]:
c = cat(tm[1,2:8],tm[2,2:8],tm[3,2:8], dims=(1,1))

21-element Vector{Any}:
  78
  28
  22
  28
  32
 164
  16
  54
    missing
  24
    missing
  52
 190
  34
  48
  28
  34
  26
  56
 198
  40

In [21]:
c = collect(skipmissing(c))

19-element Vector{Any}:
  78
  28
  22
  28
  32
 164
  16
  54
  24
  52
 190
  34
  48
  28
  34
  26
  56
 198
  40

Create a vector of the nodes. Both supplier and retailer

In [22]:
nodes = ["SA", "SB", "SC", "R1", "R2", "R3", "R4", "R5", "R6", "R7"]

10-element Vector{String}:
 "SA"
 "SB"
 "SC"
 "R1"
 "R2"
 "R3"
 "R4"
 "R5"
 "R6"
 "R7"

In [23]:
links=[]
for i in 1:3
    for j in 4:10
        if (nodes[i], nodes[j]) != ("SB", "R2") && (nodes[i], nodes[j]) != ("SB", "R4")
            push!(links, (nodes[i], nodes[j]))
        end
    end
end

We have created a vector of all the link pairs, and have excluded SB,R2 and SB,R4 since these routes are not possible. Goods can only move in one direction, from supplier to retailer.

In [24]:
Tuple(links)

(("SA", "R1"), ("SA", "R2"), ("SA", "R3"), ("SA", "R4"), ("SA", "R5"), ("SA", "R6"), ("SA", "R7"), ("SB", "R1"), ("SB", "R3"), ("SB", "R5"), ("SB", "R6"), ("SB", "R7"), ("SC", "R1"), ("SC", "R2"), ("SC", "R3"), ("SC", "R4"), ("SC", "R5"), ("SC", "R6"), ("SC", "R7"))

Next step is to create a dictionary will all the possible links and the cost of moving goods between the links

In [25]:
c_dict = Dict(links .=> c)

Dict{Tuple{String, String}, Int64} with 19 entries:
  ("SA", "R5") => 32
  ("SA", "R6") => 164
  ("SC", "R4") => 26
  ("SB", "R1") => 54
  ("SC", "R7") => 40
  ("SA", "R1") => 78
  ("SB", "R3") => 24
  ("SA", "R4") => 28
  ("SC", "R1") => 48
  ("SC", "R2") => 28
  ("SA", "R7") => 16
  ("SC", "R5") => 56
  ("SC", "R6") => 198
  ("SA", "R2") => 28
  ("SA", "R3") => 22
  ("SB", "R5") => 52
  ("SB", "R6") => 190
  ("SC", "R3") => 34
  ("SB", "R7") => 34

Next we're going to setup the optimization problem.  
The variables will be all the possible route links.  
Our objective is to minimize the cost of moving goods between suppliers and retailers i.e. sum of (number of units moved between each route) * (cost of moving a unit between the route).  
Our constraints are the total number of goods that must be removed from the supplier or moved to the retailer, respectively

In [26]:
using JuMP, GLPK

m = Model(GLPK.Optimizer)

@variable(m, 0<= x[link in links])

@objective(m, Min, sum(c_dict[link] * x[link] for link in links))

count=0

for i in nodes[1:3]
    count += 1
    @constraint(m, sum(x[(k,j)] for (k,j) in links if k == i) == supply[count])
end

count=0

for i in nodes[4:10]
    count += 1
    @constraint(m, sum(x[(k,j)] for (k,j) in links if j == i) == demand[count])
end


In [27]:
print(m)

In [28]:
JuMP.optimize!(m)

In [29]:
println("Objective value: ", JuMP.objective_value(m))

Objective value: 800000.0


In [30]:
for variable in all_variables(m)
    println("$(name(variable)) = $(value(variable))")
end

x[("SA", "R1")] = 0.0
x[("SA", "R2")] = 0.0
x[("SA", "R3")] = 0.0
x[("SA", "R4")] = 0.0
x[("SA", "R5")] = 600.0
x[("SA", "R6")] = 2200.0
x[("SA", "R7")] = 0.0
x[("SB", "R1")] = 0.0
x[("SB", "R3")] = 1200.0
x[("SB", "R5")] = 2000.0
x[("SB", "R6")] = 0.0
x[("SB", "R7")] = 2000.0
x[("SC", "R1")] = 1800.0
x[("SC", "R2")] = 2400.0
x[("SC", "R3")] = 0.0
x[("SC", "R4")] = 800.0
x[("SC", "R5")] = 800.0
x[("SC", "R6")] = 0.0
x[("SC", "R7")] = 0.0


In [31]:
for variable in all_variables(m)
    if value(variable) != 0
        println("$(name(variable)) = $(value(variable))")
    end
end

x[("SA", "R5")] = 600.0
x[("SA", "R6")] = 2200.0
x[("SB", "R3")] = 1200.0
x[("SB", "R5")] = 2000.0
x[("SB", "R7")] = 2000.0
x[("SC", "R1")] = 1800.0
x[("SC", "R2")] = 2400.0
x[("SC", "R4")] = 800.0
x[("SC", "R5")] = 800.0


##### Supplier A will supply:  
600 units to Retailer 5  
2200 units to Retailer 6  

##### Supplier B will supply:  
1200 units to Retailer 3  
2000 units to Retailer 5  
2000 units to Retailer 7  

##### Supplier C will supply:  
1800 units to Retailer 1  
2400 units to Retailer 2  
800 units to Retailer 4  
800 units to Retailer 5  

The minimum cost of moving the goods is $800,000.